In [136]:
import gradio as gr
import json
from tensorflow.keras.models import model_from_json
from tensorflow import keras
from keras.layers import Input
# Load the model architecture from a JSON file
with open('model.json', 'r') as f:
    model_json = f.read()
    f.close()

keras.backend.clear_session()
model_new = model_from_json(model_json)

# Load the trained weights from an H5 file
model_new.load_weights('model.h5')
model_new.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_new.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 189, 64)           384       
                                                                 
 conv1d_1 (Conv1D)           (None, 185, 128)          41088     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 92, 128)          0         
 )                                                               
                                                                 
 conv1d_2 (Conv1D)           (None, 88, 256)           164096    
                                                                 
 dropout (Dropout)           (None, 88, 256)           0         
                                                                 
 flatten (Flatten)           (None, 22528)             0         
                                                        

In [135]:
import librosa
import numpy as np
def model_predict(audio):
    sound, sample_rate = librosa.load(audio)
    stft = np.abs(librosa.stft(sound))

    mfccs = np.mean(librosa.feature.mfcc(y=sound, sr=sample_rate, n_mfcc=40), axis=1)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate), axis=1)
    mel = np.mean(librosa.feature.melspectrogram(y=sound, sr=sample_rate), axis=1)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate), axis=1)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(sound), sr=sample_rate), axis=1)

    concat = np.concatenate((mfccs, chroma, mel, contrast, tonnetz))
    preds = model_new.predict(np.array([np.reshape(concat, (193, 1))])) 
    preds = [round(ele * 100, 2) for ele in preds[0]]
    return dict({"COPD": preds[0],
                 "healthy": preds[1],                 "URTI": preds[2],                    "Bronchiectasis": preds[3],
                 "Pneumonia": preds[4],
                   "Bronchiolitis": preds[5]})
model_predict('sound.mp3') 
   
  

1/1 [==============================] - 0s 46ms/step


{'COPD': 100.0,
 'healthy': 0.0,
 'URTI': 0.0,
 'Bronchiectasis': 0.0,
 'Pneumonia': 0.0,
 'Bronchiolitis': 0.0}

In [132]:
audio_input = gr.Audio(type='filepath',label='Upload an audio file')
label_output = gr.Label(label='Predicted respiratory disease')

# Create the Gradio interface with the classify_respiratory_sound function as the prediction function
gradio_interface = gr.Interface(fn=model_predict, inputs=audio_input, outputs=label_output, 
                                title='Respiratory Sound Classification', 
                                description='Upload an audio file of respiratory sound to classify the respiratory disease.')

In [133]:
gradio_interface.launch()

Running on local URL:  http://127.0.0.1:7882

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 0s 65ms/step
